## 데이터 왜곡이 예측에 미치는 영향_ 타이타닉호 생존여부 머신러닝(SVM)으로 예측하기

### 1. 사용 패키지와 모듈 임포트

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split #학습데이터와 테스트데이터 분할
from sklearn import svm   #SVM 모델
from sklearn import metrics  #정확도 비교

import seaborn as sns

pd.set_option('mode.chained_assignment',  None) # Warning 방지용

### 2. 학습, 테스트 데이터 각각 가져오기
#### 데이터는 여성의 비율을 6%, 남성의 비율을 94%로 조정한 데이터 - 여성은 모든 등급좌석에서 생존율이 남성보다 높고, 1,2 등급은 월등히 높았음

In [ ]:
train_df = pd.read_csv('./titanic_train_male_large.csv')   #남성 비율이 94%인 경우
#train_df = pd.read_csv('./titanic_train_female_large.csv') #남성 비율이 14%, 여성 비율이 86%인 경우
train_df.head()

In [ ]:
test_df = pd.read_csv('./test.csv')
test_df.head()

In [ ]:
#pclass : Passenger Class, 승객등급
#name : 승객이름
#sex : 승객성별 (여성은 0, 남성은 1)
#age : 승객나이
#sibsp : 동승한 형제 또는 배우자 수
#parch : 동승한 부모 또는 자녀 수
#ticket : 티켓번호
#fare : 승객 지불 요금
#cabin : 선실이름
#embarked : 승선항(C=쉘 브르크, Q=퀸즈타운, S=사우스 햄튼)
#survived : 생존 여부(생존은 1, 아닌경우는 0)

In [ ]:
#데이터의 행,열 개수 확인
print(train_df.shape)
print(test_df.shape)

In [ ]:
#나이 데이터를 범주화 하기 위해서 10으로 나눈 몫에서 10을 곱해줌 
train_df['age'] = (train_df['age']//10) * 10  
train_df.head()

In [ ]:
train_df.corr()

In [ ]:
#나이 데이터를 범주화 하기 위해서 10으로 나눈 몫에서 10을 곱해줌 
test_df['age'] = (test_df['age']//10) * 10  
test_df.head()

### 데이터 셋을 train_test_split()로 분할할 필요없음, 이미 나누어진 데이터를 각각 가져왔음
### 성별이 결과에 큰영향이 없다고 가정하고 의도적으로 sex 변수를 학습 피쳐로 넣지 않고, pclass와 age 변수만을 선택해서 학습 모델을 만들어 보자.

### 3. 학습용 데이터셋 : 학습데이터와 레이블(정답) 나누기

In [ ]:
train_data_df = train_df[['pclass','age']]
train_data_df.head()

In [ ]:
train_label_df = train_df[['survived']]
train_label_df.head()

In [ ]:
train_data = train_data_df.values           # 값부분만 가져오기(2차원 형태)
train_data

In [ ]:
train_label = train_label_df.values.ravel() # 값부분을 1차원으로 펴주기
train_label

### 4. 테스트 데이터셋 : 학습데이터와 레이블(정답) 나누기

In [ ]:
test_data_df = test_df[['pclass','age']]
test_data_df.head()

In [ ]:
test_label_df = test_df[['survived']]  
test_label_df.head()

In [ ]:
test_data = test_data_df.values           # 값부분만 가져오기(2차원 형태)
test_data

In [ ]:
test_label = test_label_df.values.ravel() # 값부분을 1차원으로 펴주기
test_label

### 5. SVM 모델 학습하기

In [ ]:
clf = svm.SVC(C = 1, gamma = 0.1)     #svm 객체 생성
clf.fit(train_data, train_label)      #svm 모델로 학습데이터로 학습하기

### 6. 테스트 데이터로 예측하기

In [ ]:
pred_svm = clf.predict(test_data)
pred_svm  #svm이 예측한 생존 여부 값

In [ ]:
#예측 결과를 test데이터에 추가해줌
test_df['survived_predict']=pred_svm
test_df.head()

### 7. 테스트 데이터의 예측 결과 그래프로 나타내보기

In [ ]:
#테스트 데이터의 pclass 피처에서 survived_predict 피처를 기준으로 그룹을 나누어 그룹별 분포를 살펴봄
# 1은 생존자 0은 사망자

print(test_df['pclass'].value_counts()) # plclass내에서 각 그룹별 수를 세어줌
ax = sns.countplot(x='pclass', hue= 'survived_predict', data= test_df)  # pclass의 count의 수를 hue에 설정된 카테고리로 나눠 그려줌

#1등급 클래스에서 생존자가 줄어듦

In [ ]:
#테스트데이터의 survived_predict 피처를 기준으로 그룹을 나누어 그룹별 sex 피처의 분포를 살펴봄 (0은 여성, 1은 남성)
print(test_df['sex'].value_counts())
ax = sns.countplot(x='sex', hue= 'survived_predict', data= test_df)

#여성의 사망자와 남성의 생존자가 증가

In [ ]:
test_df.groupby(['pclass','sex']).mean()
#1등급 탑승객은 평균연령이 가장 높음, 1등급 클래스 여성이 가장 생존율이 높음, 2,3등급 클래스 남성이 가장 생존율이 낮음